In [1]:
import sys, os 
sys.path.append('/home/monte.flora/python_packages/frdd-wofs-cast/')

from wofscast import data_utils
from wofscast.wofscast_task_config import WOFS_TASK_CONFIG, train_lead_times
from wofscast.data_generator import to_static_vars, add_local_solar_time, load_wofscast_data

import xarray as xr
from glob import glob
import numpy as np
import dataclasses
import random 


import random
from collections import defaultdict

In [2]:
def check_for_nans(dataset):

    # Iterate through each variable in the Dataset
    for var_name, data_array in dataset.items():
        # Find boolean mask of NaNs
        nan_mask = data_array.isnull()
    
        # Use np.where to find the indices of NaNs
        nan_indices = np.where(nan_mask)
    
        # `nan_indices` is a tuple of arrays, each array corresponds to indices along one dimension
        # Print the locations of NaNs
        print(f"NaN locations in {var_name}:")
        for dim, inds in zip(nan_mask.dims, nan_indices):
            print(f"  {dim}: {inds}")   

### Randomly sample the different cases and ensemble members to improve training dataset diversity 


For the evaluation dataset, only sampling for 2021 and for the 2 hr lead times. 

In [3]:
data_paths = []
# Only doing 2019 and 2020 and leaving 2021 as the evaluation dataset?
for year in ['2021']:
    data_paths.extend(glob(os.path.join(f'/work/mflora/wofs-cast-data/datasets_2hr/{year}/wrf*.nc')))

# Function to parse date and ensemble member from a file name
def parse_file_info(file_name):
    parts = file_name.split('_')
    date = parts[1]
    ens_mem = parts[-1].split('.')[0]
    return date, ens_mem

# Organize files by date and ensemble member
files_by_date_and_ens = defaultdict(lambda: defaultdict(list))
for file_path in data_paths:
    file_name = os.path.basename(file_path)
    date, ens_mem = parse_file_info(file_name)
    files_by_date_and_ens[date][ens_mem].append(file_path)

# Decide how many samples you want per date and ensemble member
samples_per_date_and_ens = 1  # Example: 1 sample per combination

# Sample files
sampled_files = []
for date, ens_members in files_by_date_and_ens.items():
    for ens_mem, files in ens_members.items():
        if len(files) >= samples_per_date_and_ens:
            sampled_files.extend(random.sample(files, samples_per_date_and_ens))
        else:
            sampled_files.extend(files)  # Add all if fewer files than desired samples

# sampled_files now contains your randomly sampled files
print(len(sampled_files))

828


In [4]:
ds = xr.load_dataset(data_paths[0])
ds

<xarray.Dataset>
Dimensions:              (time: 14, lat: 150, lon: 150, level: 17, datetime: 14)
Coordinates:
  * lat                  (lat) float32 32.12 32.15 32.17 ... 36.14 36.17 36.19
  * lon                  (lon) float32 79.53 79.56 79.59 ... 84.12 84.16 84.19
  * time                 (time) timedelta64[ns] 00:00:00 00:10:00 ... 02:10:00
  * datetime             (datetime) datetime64[ns] 2021-04-28T21:00:00 ... 20...
  * level                (level) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
Data variables: (12/16)
    ALBEDO               (time, lat, lon) float32 0.1541 0.1539 ... 0.1318
    CLDFRA               (time, level, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
    COMPOSITE_REFL_10CM  (time, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    EMISS                (time, lat, lon) float32 0.9202 0.9199 ... 0.9299
    HGT                  (time, lat, lon) float32 705.2 705.8 ... 152.1 239.7
    ITIMESTEP            (time) int32 240 280 320 360 400 ... 640 680 720 760
    ...                   ...
    UP_HELI_MAX          (time, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    V                    (time, level, lat, lon) float32 3.244 3.199 ... 0.4699
    W                    (time, level, lat, lon) float32 0.0006772 ... -0.0283
    XLAND                (time, lat, lon) float32 1.0 1.0 1.0 ... 1.0 1.0 1.0
    GEOPOT               (time, level, lat, lon) float32 7.004e+03 ... 2.415e+05
    RAIN_AMOUNT          (lat, lon, time) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/74)
    TITLE:                            OUTPUT FROM WRF V3.9pre#2 MODEL
    START_DATE:                      2021-04-28_20:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      301
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              3000.0
    ...                              ...
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0
    history:                         Wed May 12 01:05:31 2021: /scratch/softw...
    NCO:                             4.6.7

### Creating the evaluation dataset

For the evaluation dataset, only sampling from a single ensemble member on each date. 

In [5]:
from tqdm.notebook import tqdm
def read_netcdfs(paths, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds
        
    datasets = [process_one_path(p) for p in tqdm(paths, desc="Processing files")]
    combined = xr.concat(datasets, dim)
    return combined

In [6]:
%%time
# Since the 2 hr datasets are large, only using 256 samples for evaluation. 
dataset = read_netcdfs(sampled_files[:256], dim='batch',
                        transform_func=add_local_solar_time)

inputs, targets, forcings = data_utils.extract_inputs_targets_forcings(dataset,
                                                        target_lead_times=slice('10min', '120min'),
                                                        **dataclasses.asdict(WOFS_TASK_CONFIG))

Processing files:   0%|          | 0/256 [00:00<?, ?it/s]

CPU times: user 3min 5s, sys: 2min 57s, total: 6min 3s
Wall time: 12min 59s


In [7]:
print('Eval Input Dims: ', inputs.dims.mapping)
print('Eval Target Dims: ', targets.dims.mapping)
print('Eval Forcing Dims: ', forcings.dims.mapping)

Eval Input Dims:  {'batch': 256, 'time': 2, 'level': 17, 'lat': 150, 'lon': 150}
Eval Target Dims:  {'batch': 256, 'time': 12, 'level': 17, 'lat': 150, 'lon': 150}
Eval Forcing Dims:  {'batch': 256, 'time': 12, 'lon': 150, 'lat': 150}


In [8]:
# Calculate the memory usage in bytes
memory_usage_bytes = targets.nbytes + inputs.nbytes + forcings.nbytes

# Alternatively, for gigabytes (GB)
memory_usage_gb = memory_usage_bytes / (1024**3)
print(f"Total Memory Usage for Inputs, Targets, and Forcings: {memory_usage_gb:.2f} GB")

Total Memory Usage for Inputs, Targets, and Forcings: 32.96 GB


In [9]:
%%time
out_path = '/work/mflora/wofs-cast-data/eval_datasets'
inputs.to_netcdf(os.path.join(out_path, 'eval_inputs.nc'))
targets.to_netcdf(os.path.join(out_path, 'eval_targets.nc'))
forcings.to_netcdf(os.path.join(out_path, 'eval_forcings.nc'))

CPU times: user 18min 35s, sys: 16.1 s, total: 18min 51s
Wall time: 19min 2s


In [10]:
import gc
gc.collect()

81